# Navigation

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [2]:
!pip -q install ./python

tensorflow 1.7.1 has requirement numpy>=1.13.3, but you'll have numpy 1.12.1 which is incompatible.
ipython 6.5.0 has requirement prompt-toolkit<2.0.0,>=1.0.15, but you'll have prompt-toolkit 2.0.10 which is incompatible.


The environment is already saved in the Workspace and can be accessed at the file path provided below.  Please run the next code cell without making any changes.

In [3]:
import torch
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from params import VarParam, DQNParameters
from agent import Agent
from unityagents import UnityEnvironment
from collections import deque


# please do not modify the line below
env = UnityEnvironment(file_name="/data/Banana_Linux_NoVis/Banana.x86_64")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [4]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [5]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)

Number of agents: 1
Number of actions: 4
States look like: [ 1.          0.          0.          0.          0.84408134  0.          0.
  1.          0.          0.0748472   0.          1.          0.          0.
  0.25755     1.          0.          0.          0.          0.74177343
  0.          1.          0.          0.          0.25854847  0.          0.
  1.          0.          0.09355672  0.          1.          0.          0.
  0.31969345  0.          0.        ]
States have length: 37


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Note that **in this coding environment, you will not be able to watch the agent while it is training**, and you should set `train_mode=True` to restart the environment.

In [6]:
env_info = env.reset(train_mode=True)[brain_name] # reset the environment
state = env_info.vector_observations[0]            # get the current state
score = 0                                          # initialize the score
for i in range(100):
    action = np.random.randint(action_size)        # select an action
    env_info = env.step(action)[brain_name]        # send the action to the environment
    next_state = env_info.vector_observations[0]   # get the next state
    reward = env_info.rewards[0]                   # get the reward
    done = env_info.local_done[0]                  # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    if done:                                       # exit loop if episode finished
        break
    
print("Score: {}".format(score))

Score: 0.0


When finished, you can close the environment.

In [7]:
# env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  A few **important notes**:
- When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```
- To structure your work, you're welcome to work directly in this Jupyter notebook, or you might like to start over with a new file!  You can see the list of files in the workspace by clicking on **_Jupyter_** in the top left corner of the notebook.
- In this coding environment, you will not be able to watch the agent while it is training.  However, **_after training the agent_**, you can download the saved model weights to watch the agent on your own machine! 

### 5. Let us train an agent using DQN in the Banana Environment

The code cell below has the function that will train the agent.


In [8]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space
state = env_info.vector_observations[0]
state_size = len(state)
print('States have length:', state_size)

params = DQNParameters(state_size=state_size, action_size=action_size)
agent = Agent(params)


Number of actions: 4
States have length: 37


In [ ]:
n_episodes=2000                        # Total number of episodes to train on
max_t=100000                           # Maximum number of actions to take within an episiode 
passing_score_achieved=False           # Achieved when average score for 100 episodes reaches 13
scores = []                            # List of scores for an episode 
scores_window = deque(maxlen=100)      # Most recent 100 scores

for i_episode in range(1, n_episodes + 1):
    # Reset environment
    env_info = env.reset(train_mode=True)[brain_name]

    # Get next state
    state = env_info.vector_observations[0]

    score = 0
    for t in range(max_t):

        # Get action
        action = agent.act(state, params.epsilon())

        # Send action to environment
        env_info = env.step(action)[brain_name]

        # Get next state
        next_state = env_info.vector_observations[0]

        # Get reward
        reward = env_info.rewards[0]

        # Check is episode finished
        done = env_info.local_done[0]

        # run agent step - which adds to the experience and also learns based on UPDATE_EVERY
        agent.step(state, action, reward, next_state, done, params.gamma(), params.beta(), params.tau())

        score += reward
        state = next_state

        # Exit if episode finished
        if done:
            break

    scores_window.append(score)
    scores.append(score)

    params.epsilon.decrement()   # reduce randmoness for action selection
    params.gamma.increment()     # give more priority to future returns
    params.tau.increment()       # make larger updates to target DQN
    params.beta.increment()      # increase bias for weights from stored experiences

    print('\rEpisode {}\tAverage Score: {:.4f}\t{}'.format(i_episode, np.mean(scores_window), params), end="")

    if i_episode % scores_window.maxlen == 0:
        print('\r\nEpisode {}\tAverage Score: {:.4f}'.format(i_episode, np.mean(scores_window)))
    if np.mean(scores_window) > 13 and not passing_score_achieved:
        passing_score_achieved = True
        print('\rPassing score achieved in {:d} episodes!\tAverage Score: {:.4f}'.format(i_episode, np.mean(scores_window)))

if passing_score_achieved:
    torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.pth')
else:
    print("Passing score not achieved :(")

# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

Episode 100	Average Score: 0.5300	lr=0.0005, epsilon=0.610, gamma=0.957, beta=0.637, tau=0.001
Episode 200	Average Score: 4.1500	lr=0.0005, epsilon=0.373, gamma=0.978, beta=0.780, tau=0.001
Episode 300	Average Score: 7.4200	lr=0.0005, epsilon=0.230, gamma=0.986, beta=0.867, tau=0.001
Episode 400	Average Score: 9.8800	lr=0.0005, epsilon=0.143, gamma=0.988, beta=0.919, tau=0.001
Episode 500	Average Score: 10.6500	lr=0.0005, epsilon=0.091, gamma=0.989, beta=0.951, tau=0.001
Episode 600	Average Score: 11.7200	lr=0.0005, epsilon=0.059, gamma=0.990, beta=0.970, tau=0.002
Episode 700	Average Score: 12.2900	lr=0.0005, epsilon=0.040, gamma=0.990, beta=0.982, tau=0.002
Passing score achieved in 718 episodes!	Average Score: 13.1000amma=0.990, beta=0.984, tau=0.002
Episode 800	Average Score: 14.1000	lr=0.0005, epsilon=0.028, gamma=0.990, beta=0.989, tau=0.002
Episode 900	Average Score: 13.7000	lr=0.0005, epsilon=0.021, gamma=0.990, beta=0.993, tau=0.002
Episode 1000	Average Score: 13.4700	lr=0.000

### 6. Let us see how a trained agent performs.

Let us see how a trained agent scores. Note: You will NOT be able to actually see the environment in this workspace.

In [11]:
# load the weights from file
agent.qnetwork_local.load_state_dict(torch.load('checkpoint.pth'))
brain_name = env.brain_names[0]

n_episodes=100
max_t=100000

total_score = 0

for i in range(n_episodes):

    # Reset the environment. Training mode is off.
    env_info = env.reset(train_mode=False)[brain_name]

    # Get the current state
    state = env_info.vector_observations[0]

    score = 0

    for t in range(max_t):
        # Decide on an action given the current state
        action = agent.act(state)

        # Send action to the environment
        env_info = env.step(action)[brain_name]

        # Get the next state
        next_state = env_info.vector_observations[0]

        # Get the reward
        reward = env_info.rewards[0]

        # Check if the episode is finished
        done = env_info.local_done[0]

        # Add the current reward into the score
        score += reward

        state = next_state

        # Exit the loop when the episode is done
        if done:
            break
    total_score += score

    print("\rTest run[{}]\tscore: {}".format(i, score), end="")

print("\rAverage Score: {}".format(total_score / n_episodes))

Score: 20.0
Score: 12.0
Score: 0.0
Score: 10.0
Score: 1.0
Score: 0.0
Score: 1.0
Score: 15.0
Score: 3.0
Score: 27.0
Average Score: 0.0445
